# Environment Set Up

In [1]:
! pip install awswrangler

     |████████████████████████████████| 183 kB 27.8 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 2.5 MB/s  eta 0:00:01
     |████████████████████████████████| 43 kB 4.8 MB/s  eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import awswrangler as wr
import pandas as pd

# Create Athena Tables

Data downloaded from Ministero delle Finanze: https://www1.finanze.gov.it/finanze3/analisi_stat/index.php?search_class[0]=cCOMUNE&opendata=yes

### 1) Tax Declarations Comuni

In [ ]:
comuni_path = 'Redditi_e_principali_variabili_IRPEF_su_base_comunale_CSV_2019.csv'
comuni = pd.read_csv(comuni_path, sep = ";")

# Fixing columns empty names issues
cols = comuni.columns[:-1]
new_comuni = comuni.reset_index().iloc[:,:-2]
new_comuni.columns = cols

In [41]:
# Uploading to Athena and S3
wr.s3.to_parquet(
    df=new_comuni,
    path="s3://gimi-data/out/italy/tax-declarations/tax-declarations-comuni/",
    dataset=True,
    database="gimi",
    table="irpef-comuni",
)

{'paths': ['s3://gimi-data/out/italy/tax-declarations/tax-declarations-comuni/4a3bc2c9dc814c18a78bdc03f676d231.snappy.parquet'],
 'partitions_values': {}}

### 1) Tax Declarations Comuni Sub (Cap level for cities)

In [47]:
comuni_sub_path = 'Redditi_e_principali_variabili_IRPEF_su_base_subcomunale_CSV_2019.csv'
comuni_sub = pd.read_csv(comuni_path, sep = ";")

# Fixing columns empty names issues
cols = comuni_sub.columns[:-1]
new_comuni_sub = comuni_sub.reset_index().iloc[:,:-2]
new_comuni_sub.columns = cols

In [57]:
# Uploading to Athena and S3
wr.s3.to_parquet(
    df=new_comuni_sub,
    path="s3://gimi-data/out/italy/tax-declarations/tax-declarations-comuni-sub/",
    dataset=True,
    database="gimi",
    table="irpef-comuni-sub",
)

{'paths': ['s3://gimi-data/out/italy/tax-declarations/tax-declarations-comuni-sub/fe26920be8334e0fb72148b036a8577c.snappy.parquet'],
 'partitions_values': {}}

In [53]:
# Sample Row cap level
sub = new_comuni_sub[new_comuni_sub['CAP'] == 20144]
sub.T

,250
Anno di imposta,2019
CAP,20144
Codice catastale,F205
Codice Istat Comune,15146
Denominazione Comune,MILANO
Sigla Provincia,MI
Regione,Lombardia
Codice Istat Regione,3
Numero contribuenti,26697
Reddito da fabbricati - Frequenza,13987
